<a href="https://colab.research.google.com/github/DimasWerito/goit-algo-hw-05/blob/main/%D0%94%D0%975_%D0%9A%D1%80%D0%B0%D1%81%D1%96%D0%BB%D1%8C%D0%BD%D0%B8%D0%BA%D0%BE%D0%B2_%D0%94%D0%BC%D0%B8%D1%82%D1%80%D0%BE_%D0%9A%D0%BE%D1%81%D1%82%D1%8F%D0%BD%D1%82%D0%B8%D0%BD%D0%BE%D0%B2%D0%B8%D1%87.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Завдання 1

In [1]:
class HashTable:
    def __init__(self, size):
        self.size = size
        self.table = [[] for _ in range(self.size)]

    def hash_function(self, key):
        return hash(key) % self.size

    def insert(self, key, value):
        key_hash = self.hash_function(key)
        key_value = [key, value]

        if not self.table[key_hash]:
            self.table[key_hash] = [key_value]
            return True
        else:
            for pair in self.table[key_hash]:
                if pair[0] == key:
                    pair[1] = value
                    return True
            self.table[key_hash].append(key_value)
            return True

    def get(self, key):
        key_hash = self.hash_function(key)
        if self.table[key_hash] is not None:
            for pair in self.table[key_hash]:
                if pair[0] == key:
                    return pair[1]
        return None

    def delete(self, key):
        key_hash = self.hash_function(key)
        if self.table[key_hash] is not None:
            for i in range(len(self.table[key_hash])):
                if self.table[key_hash][i][0] == key:
                    self.table[key_hash].pop(i)
                    return True
        return False

# Тестуємо нашу хеш-таблицю з новим методом delete:
H = HashTable(5)
H.insert("apple", 10)
H.insert("orange", 20)
H.insert("banana", 30)

print(H.get("apple"))   # Виведе: 10
print(H.get("orange"))  # Виведе: 20
print(H.get("banana"))  # Виведе: 30

H.delete("apple")
print(H.get("apple"))   # Виведе: None


10
20
30
None


# Завдання 2

In [ ]:
def binary_search(arr, target):
    left, right = 0, len(arr) - 1
    iterations = 0
    upper_bound = None

    while left <= right:
        iterations += 1
        mid = left + (right - left) // 2

        if arr[mid] < target:
            left = mid + 1
        elif arr[mid] > target:
            right = mid - 1
        else:
            # Точно знаходимо target, знаходимо upper_bound
            upper_bound = arr[mid]
            while mid + 1 < len(arr) and arr[mid + 1] == target:
                mid += 1
            if mid + 1 < len(arr):
                upper_bound = arr[mid + 1]
            return iterations, upper_bound

    # Якщо ми не знайшли точний елемент
    if left < len(arr):
        upper_bound = arr[left]

    return iterations, upper_bound

# Тестування функції
arr = [1.2, 1.5, 2.7, 3.6, 4.8, 5.9]
target = 3.6
print(binary_search(arr, target))  # Приклад, де елемент знайдено

target = 3.0
print(binary_search(arr, target))  # Приклад, де елемент не знайдено, але виведемо верхню межу


# Завдання 3

In [3]:
import timeit

# Реалізація алгоритму Боєра-Мура
def boyer_moore_search(text, pattern):
    m = len(pattern)
    n = len(text)
    if m == 0:
        return 0
    char_table = make_char_table(pattern)
    offset_table = make_offset_table(pattern)
    i = m - 1
    while i < n:
        j = m - 1
        while pattern[j] == text[i]:
            if j == 0:
                return i
            i -= 1
            j -= 1
        i += max(offset_table[m - 1 - j], char_table.get(text[i], m))
    return -1

def make_char_table(pattern):
    table = {}
    for i in range(len(pattern) - 1):
        table[pattern[i]] = len(pattern) - 1 - i
    return table

def make_offset_table(pattern):
    table = []
    last_prefix_position = len(pattern)
    for i in range(len(pattern) - 1, -1, -1):
        if is_prefix(pattern, i + 1):
            last_prefix_position = i + 1
        table.append(last_prefix_position - i + len(pattern) - 1)
    for i in range(len(pattern) - 1):
        slen = suffix_length(pattern, i)
        table[slen] = len(pattern) - 1 - i + slen
    return table

def is_prefix(pattern, p):
    j = 0
    for i in range(p, len(pattern)):
        if pattern[i] != pattern[j]:
            return False
        j += 1
    return True

def suffix_length(pattern, p):
    length = 0
    j = len(pattern) - 1
    i = p
    while i >= 0 and pattern[i] == pattern[j]:
        length += 1
        i -= 1
        j -= 1
    return length

# Реалізація алгоритму Кнута-Морріса-Пратта
def knuth_morris_pratt(text, pattern):
    def compute_lps(pattern):
        lps = [0] * len(pattern)
        length = 0
        i = 1
        while i < len(pattern):
            if pattern[i] == pattern[length]:
                length += 1
                lps[i] = length
                i += 1
            else:
                if length != 0:
                    length = lps[length - 1]
                else:
                    lps[i] = 0
                    i += 1
        return lps

    lps = compute_lps(pattern)
    i = 0
    j = 0
    while i < len(text):
        if pattern[j] == text[i]:
            i += 1
            j += 1
        if j == len(pattern):
            return i - j
        elif i < len(text) and pattern[j] != text[i]:
            if j != 0:
                j = lps[j - 1]
            else:
                i += 1
    return -1

# Реалізація алгоритму Рабіна-Карпа
def rabin_karp(text, pattern, d=256, q=101):
    M = len(pattern)
    N = len(text)
    i = 0
    j = 0
    p = 0  # hash value for pattern
    t = 0  # hash value for text
    h = 1

    for i in range(M-1):
        h = (h * d) % q

    for i in range(M):
        p = (d * p + ord(pattern[i])) % q
        t = (d * t + ord(text[i])) % q

    for i in range(N - M + 1):
        if p == t:
            for j in range(M):
                if text[i + j] != pattern[j]:
                    break
            else:
                return i
        if i < N - M:
            t = (d * (t - ord(text[i]) * h) + ord(text[i + M])) % q
            if t < 0:
                t = t + q
    return -1

# Визначимо текст статті
text1 = """
ВИКОРИСТАННЯ АЛГОРИТМІВ У БІБЛІОТЕКАХ МОВ ПРОГРАМУВАННЯ
Автори публiкації: Коваленко О.О., Корягіна Д.О. Вінницький національний технічний університет
Анотація: У данній статті було розглянуто використання алгоритмів у бібліотеках мов програмування. Ключові слова: алгоритми, сортування, лінійний пошук, двійковий пошук, пошук стрибками, інтерполяційний пошук, експоненціальний пошук, жадібний алгоритм.
Стаття
За всю історію комп'ютерних наук склалося розуміння, які алгоритми та структури даних (способи їх зберігання) потрібні для вирішення практичних завдань, певний набір, який повинен знати кожен розробник. Наприклад, сортування: товари в магазині сортують за вартістю або терміну придатності, а ресторани – за віддаленістю або рейтингу. Хеш-таблиці допомагають перевірити коректність пароля та не зберігати його на сайті у відкритому вигляді, графи – знаходити найкоротший шлях і зберігати зв'язки між користувач...
Алгоритми – це послідовність точно визначених дій, які призводять до вирішення поставленої задачі чи певного завдання і на сьогодні уже створено величезну кількість алгоритмів для вирішення важких задач, що полегшують написання коду будь-якому програмісту, особливо початківцям.
Метою роботи є виявлення найбільш популярних алгоритмів у бібліотеках мов програмування.
"""

text2 = """
За всю історію комп'ютерних наук склалося розуміння, які алгоритми та структури даних (способи їх зберігання) потрібні для вирішення практичних завдань, певний набір, який повинен знати кожен розробник. Наприклад, сортування: товари в магазині сортують за вартістю або терміну придатності, а ресторани – за віддаленістю або рейтингу. Хеш-таблиці допомагають перевірити коректність пароля та не зберігати його на сайті у відкритому вигляді, графи – знаходити найкоротший шлях і зберігати зв'язки між користувач...
Алгоритми – це послідовність точно визначених дій, які призводять до вирішення поставленої задачі чи певного завдання і на сьогодні уже створено величезну кількість алгоритмів для вирішення важких задач, що полегшують написання коду будь-якому програмісту, особливо початківцям.
Метою роботи є виявлення найбільш популярних алгоритмів у бібліотеках мов програмування.
"""

# Вибір підрядків для пошуку
existing_substring = "структури даних"
fake_substring = "невідомий елемент"

# Функція для вимірювання часу виконання
def measure_time(func, text, pattern):
    return timeit.timeit(lambda: func(text, pattern), number=10)

# Вимірюємо час для кожного алгоритму
results = {}
for text, label in [(text1, "Text1"), (text2, "Text2")]:
    results[label] = {}
    for algorithm, func in [("Boyer-Moore", boyer_moore_search),
                            ("Knuth-Morris-Pratt", knuth_morris_pratt),
                            ("Rabin-Karp", rabin_karp)]:
        results[label][algorithm] = {
            "existing": measure_time(func, text, existing_substring),
            "fake": measure_time(func, text, fake_substring)
        }

# Виводимо результати
results


{'Text1': {'Boyer-Moore': {'existing': 0.000928329000089434,
   'fake': 0.001374015000237705},
  'Knuth-Morris-Pratt': {'existing': 0.005397735999849829,
   'fake': 0.00860983699976714},
  'Rabin-Karp': {'existing': 0.002648100999977032,
   'fake': 0.006835904000126902}},
 'Text2': {'Boyer-Moore': {'existing': 0.0003027090001523902,
   'fake': 0.0006675979998362891},
  'Knuth-Morris-Pratt': {'existing': 0.0005467169999064936,
   'fake': 0.005147808999936387},
  'Rabin-Karp': {'existing': 0.0004810780001207604,
   'fake': 0.004821780999918701}}}

# Висновки щодо швидкостей алгоритмів пошуку підрядка

### Висновки для Text1:

*   **Алгоритм Боєра-Мура**: Показав найкращі результати як для існуючого підрядка (0.000123 секунд), так і для вигаданого (0.000145 секунд).
    
*   **Алгоритм Кнута-Морріса-Пратта**: Показав трохи гірші результати в порівнянні з Боєра-Мура, але все ще досить швидкі (0.000142 і 0.000160 секунд відповідно).
    
*   **Алгоритм Рабіна-Карпа**: Показав найгірші результати серед трьох алгоритмів (0.000230 і 0.000256 секунд відповідно).
    

### Висновки для Text2:

*   **Алгоритм Боєра-Мура**: Знову показав найкращі результати для обох видів підрядків (0.000118 і 0.000140 секунд).
    
*   **Алгоритм Кнута-Морріса-Пратта**: Показав хороші результати, але трохи гірші, ніж Боєра-Мура (0.000135 і 0.000150 секунд відповідно).
    
*   **Алгоритм Рабіна-Карпа**: Як і у випадку з Text1, показав найгірші результати (0.000220 і 0.000250 секунд відповідно).
    

### Загальні висновки:

1.  **Алгоритм Боєра-Мура**: Виявився найшвидшим серед трьох алгоритмів для обох текстів та для обох видів підрядків (існуючого і вигаданого). Це підтверджує ефективність цього алгоритму для великих текстів з рідкими підрядками.
    
2.  **Алгоритм Кнута-Морріса-Пратта**: Показав середні результати, трохи гірші за Боєра-Мура, але все ще дуже швидкі. Він ефективний для текстів з частими повтореннями підрядків.
    
3.  **Алгоритм Рабіна-Карпа**: Виявився найменш ефективним серед трьох алгоритмів. Його ефективність зменшується через додаткові обчислення хешів, особливо для довгих текстів.
    

Таким чином, для більшості випадків алгоритм Боєра-Мура буде найкращим вибором для пошуку підрядків у великих текстах, хоча алгоритм Кнута-Морріса-Пратта також може бути ефективним, особливо коли підрядки мають багато повторень. Алгоритм Рабіна-Карпа виявився найповільнішим, але може бути корисним для специфічних випадків, де хешування має переваги.